In [4]:
#import required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [7]:
# Import required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read dataset
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Tokenizer process
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text.split('\n'))

# Assign length of word index (total number of words)
total_words = len(tokenizer.word_index) + 1  # +1 for padding token

# Declare n-grams
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Maximum sentence length
max_sequence_len = max([len(seq) for seq in input_sequences])
print(f"Max Sequence Length: {max_sequence_len}")

# Pad the sequences to have the same length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Split the input and labels
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Convert labels to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Create the model
# model = Sequential()
# model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
# model.add(LSTM(150))
# model.add(Dense(total_words, activation='softmax'))

# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Print the model summary
# print(model.summary())

# Create the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))  # Embedding layer
model.add(LSTM(150))  # LSTM layer
model.add(Dense(total_words, activation='softmax'))  # Output layer with softmax

# Build the model with a specified input shape (fix for "unbuilt" issue)
model.build(input_shape=(None, max_sequence_len-1))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
print(model.summary())


Max Sequence Length: 18


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 17, 100)        │       820,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8200)           │     1,238,200 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,208,800 (8.43 MB)

 Trainable params: 2,208,800 (8.43 MB)

 Non-trainable params: 0 (0.00 B)

None


In [18]:
#compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#fit the model
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 47s 15ms/step - accuracy: 0.3376 - loss: 3.2055
Epoch 2/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 44s 15ms/step - accuracy: 0.3736 - loss: 2.9604
Epoch 3/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.4202 - loss: 2.6990
Epoch 4/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 44s 15ms/step - accuracy: 0.4631 - loss: 2.4778
Epoch 5/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 44s 15ms/step - accuracy: 0.4940 - loss: 2.3161
Epoch 6/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.5286 - loss: 2.1495
Epoch 7/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.5552 - loss: 2.0096
Epoch 8/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.5830 - loss: 1.8896
Epoch 9/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.6080 - loss: 1.7695
Epoch 10/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.6321 - loss: 1.6638
Epoch 11/50
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.6464 - loss: 1.5807
Epoch 12

In [30]:
#determine a text
seed_text = "When you raise your cry of fire,"
# predict word number
next_words = 5

for _ in range(next_words):
    #convert to token
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    #path sequences
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    #model prediction
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    # get predict words
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


In [31]:
seed_text

'It is nothing very formidable, from self papers to do'